In [32]:
from transformers import BertConfig, BertModel, AutoTokenizer, TrainingArguments, AutoModelForSequenceClassification, Trainer, DataCollatorWithPadding
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import torch
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict, ClassLabel
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


In [33]:
torch.cuda.is_available()

True

In [34]:
tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")

In [35]:
# Loading dataset
dataSetPath = "yle_data/2011-2018-SubjectsInPaper-1.csv"
df = pd.read_csv(dataSetPath, sep="␞", nrows=200000)


print(df['text'].isna().sum(), "invalid data")
df = df.dropna(subset=['text'])
print(len(df), "valid data")

print(df.columns)
df = df.drop(['id', 'url', 'headline', 'datePublished'], axis=1) 


def multiLabelToSingular(labelArray):
    cleaned_text = labelArray.replace("'", "").replace(" ", "").replace("[", "").replace("]", "")
    return cleaned_text.split(",")[0]


# print(df.dtypes)
df['subjects'] = df['subjects'].apply(multiLabelToSingular)

print(df['subjects'].nunique(), "categories")
print(df['subjects'].unique().tolist())

df.rename({'subjects': 'labels'}, axis="columns", inplace=True)



C:\Users\joakim\AppData\Local\Temp\ipykernel_19452\3043923669.py:3: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(dataSetPath, sep="␞", nrows=200000)


612 invalid data
199388 valid data
Index(['id', 'url', 'headline', 'text', 'subjects', 'datePublished'], dtype='object')
10 categories
['urheilu', 'kulttuuri', 'talous', 'luonto', 'Onnettomuudet', 'politiikka', 'Rikokset', 'Liikenne_ja_kuljetus', 'Koulutus_ja_kasvatus', 'Terveys']


In [36]:
# paperin luokat:

# urheilu "18-220090",
# talous "18-204933",
# politiikka "18-220306",
# kulttuuri "18-208149",
# luonto "18-215452",
# 18-218480 - Onnettomuudet
# 18-209306 - Terveys
# 18-217206 - Rikokset
# 18-91232 - Liikenne ja kuljetus
# 18-35286 - Koulutus ja kasvatus

print(df['labels'].value_counts().sort_index(ascending=True))

le = preprocessing.LabelEncoder()
le.fit(df.labels)
df['labels'] = le.transform(df.labels)

Koulutus_ja_kasvatus    10753
Liikenne_ja_kuljetus    13554
Onnettomuudet           10717
Rikokset                 8166
Terveys                  8600
kulttuuri               13794
luonto                  10994
politiikka              12961
talous                  24427
urheilu                 85422
Name: labels, dtype: int64


In [37]:
n_sample = 75
df_balanced = df.groupby('labels').head(n_sample)
df_balanced = df.groupby('labels').apply(lambda x: x.sample(n=n_sample))

print(len(df_balanced), "documents")
print(df_balanced['labels'].value_counts().sort_index(ascending=True))


750 documents
0    75
1    75
2    75
3    75
4    75
5    75
6    75
7    75
8    75
9    75
Name: labels, dtype: int64


In [38]:
dataset = Dataset.from_pandas(df_balanced, preserve_index=False)
dataset

Dataset({
    features: ['text', 'labels'],
    num_rows: 750
})

In [39]:

train_test_split = dataset.train_test_split(test_size=0.2)

train_valid_split = train_test_split['train'].train_test_split(test_size=0.1)

datasets = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test'],
    'test': train_test_split['test']
})
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 540
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 60
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 150
    })
})


In [40]:
unique_values = set(datasets["train"]["labels"])
print(unique_values)


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [41]:
def tokenize_function(example):
    tokenized = tokenizer(example["text"], truncation=True)
    return tokenized

In [42]:

tokenized_datasets = datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets



Map:   0%|          | 0/540 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 540
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 60
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 150
    })
})

In [43]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": accuracy, "f1": f1}

training_args = TrainingArguments(
    output_dir="./logs",                    
    evaluation_strategy="steps",        
    eval_steps=50,                      
    logging_dir="./logs",                
    logging_steps=50,                   
    save_steps=500,                      
    per_device_train_batch_size=8,       
    per_device_eval_batch_size=16,       
    num_train_epochs=5,                  
    save_total_limit=2,                  
    load_best_model_at_end=True,         
    metric_for_best_model="f1",          
    greater_is_better=True,              
    report_to="tensorboard",            
)





In [44]:
model = AutoModelForSequenceClassification.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1", num_labels=11, device_map = 'cuda')


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\joakim\anaconda3\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [45]:
trainer.train()

  0%|          | 0/340 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.5772, 'learning_rate': 4.2647058823529415e-05, 'epoch': 0.74}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.6621029376983643, 'eval_accuracy': 0.8, 'eval_f1': 0.7908524808524808, 'eval_runtime': 10.3844, 'eval_samples_per_second': 5.778, 'eval_steps_per_second': 0.385, 'epoch': 0.74}
{'loss': 0.6307, 'learning_rate': 3.529411764705883e-05, 'epoch': 1.47}


...

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.4912361800670624, 'eval_accuracy': 0.85, 'eval_f1': 0.8467929292929294, 'eval_runtime': 10.0842, 'eval_samples_per_second': 5.95, 'eval_steps_per_second': 0.397, 'epoch': 3.68}
{'loss': 0.0874, 'learning_rate': 5.882352941176471e-06, 'epoch': 4.41}


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 0.3343992829322815, 'eval_accuracy': 0.8833333333333333, 'eval_f1': 0.8800566100566101, 'eval_runtime': 10.1574, 'eval_samples_per_second': 5.907, 'eval_steps_per_second': 0.394, 'epoch': 4.41}
{'train_runtime': 1634.1956, 'train_samples_per_second': 1.652, 'train_steps_per_second': 0.208, 'train_loss': 0.4426063977620181, 'epoch': 5.0}


TrainOutput(global_step=340, training_loss=0.4426063977620181, metrics={'train_runtime': 1634.1956, 'train_samples_per_second': 1.652, 'train_steps_per_second': 0.208, 'train_loss': 0.4426063977620181, 'epoch': 5.0})

In [46]:
predictions = trainer.predict(test_dataset=tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)


  0%|          | 0/10 [00:00<?, ?it/s]

(150, 11) (150,)


In [47]:

preds = np.argmax(predictions.predictions, axis=-1)
print(preds)
print(predictions.label_ids)


[8 6 3 3 1 1 4 3 0 6 9 0 1 1 7 1 9 0 3 1 5 0 0 2 5 2 2 3 7 0 5 6 3 0 7 4 3
 1 3 0 0 1 6 7 7 4 5 7 4 8 1 4 7 0 6 1 5 0 5 3 8 9 7 5 4 5 9 3 2 0 0 2 8 8
 7 3 1 1 2 4 5 1 7 9 1 9 9 7 7 7 0 7 1 5 5 7 7 1 0 5 9 5 2 7 9 8 5 1 0 5 3
 7 6 2 6 7 7 1 4 1 2 8 5 5 2 9 8 1 9 9 8 4 3 0 7 7 8 7 5 2 6 2 2 1 3 8 2 5
 1 0]
[8 6 3 3 8 8 4 3 0 6 9 0 1 1 8 1 9 0 3 1 5 0 0 2 5 2 2 3 7 0 5 6 3 0 7 4 3
 1 3 0 0 1 6 4 5 4 5 7 4 8 1 4 7 0 6 1 5 0 5 3 8 9 7 5 4 5 9 8 6 0 0 2 8 8
 5 3 1 6 2 4 5 1 8 9 1 9 9 7 7 8 0 7 1 5 5 7 8 1 0 5 9 5 2 7 9 6 5 1 0 5 3
 7 6 2 6 7 7 1 6 1 2 8 5 5 2 9 8 1 9 9 8 4 3 4 7 7 8 7 6 2 6 2 6 1 3 8 1 5
 1 0]


In [48]:
trainer.evaluate(tokenized_datasets["test"])



  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6164527535438538,
 'eval_accuracy': 0.88,
 'eval_f1': 0.8742900924936013,
 'eval_runtime': 24.7746,
 'eval_samples_per_second': 6.055,
 'eval_steps_per_second': 0.404,
 'epoch': 5.0}

In [49]:
model_save_path = "./t_model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)


('./t_model\\tokenizer_config.json',
 './t_model\\special_tokens_map.json',
 './t_model\\vocab.txt',
 './t_model\\added_tokens.json',
 './t_model\\tokenizer.json')

In [50]:
model_save_path = "./t_model"

model = AutoModelForSequenceClassification.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

# trainer = Trainer(
#     model=model,
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.evaluate(tokenized_datasets["test"])

c:\Users\joakim\anaconda3\lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6164527535438538,
 'eval_accuracy': 0.88,
 'eval_f1': 0.8742900924936013,
 'eval_runtime': 10.1223,
 'eval_samples_per_second': 14.819,
 'eval_steps_per_second': 0.988}